# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

import torch
from torch.utils.data import Dataset

from transformers import DistilBertTokenizerFast, BertTokenizer
from transformers import Trainer, TrainingArguments

import nltk, random
from nltk.corpus import movie_reviews

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

from lime.lime_text import LimeTextExplainer
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.layers import *
from keras.models import Sequential, Model
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils.vis_utils import plot_model
from keras.utils.vis_utils import model_to_dot

In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
import math

import os

seed_value = 42
os.environ['PYTHONHASHSEED']= str(seed_value)

import random as rn

np.random.seed(seed_value)
rn.seed(seed_value)


# Creating a BERT Tokenizer

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
vocab_size = len(tokenizer.vocab) + 1
print(vocab_size)

In [ ]:
###################Importing Train Dataset##########################

train = pd.read_csv("Preprocess_Train.csv", header=0)
train = train[train['text'].notnull()]
train = train[train['target'].notnull()]
train.head()

In [ ]:
###################Split Train Dataset to Xtrain & Ytrain##########################

Xtrain = train['text']
Ytrain = train['target']

In [ ]:
###################Tokenizing & Padding of Train Dataset##########################

maxlen = 100

Xtrain = '[CLS]' + Xtrain + '[SEP]'
Xtrain = list(map(tokenizer.tokenize, Xtrain))

c = Xtrain

Xtrain = [tokenizer.convert_tokens_to_ids(txt) for txt in Xtrain]
Xtrain = pad_sequences(Xtrain, padding='post', maxlen=maxlen)
Xtrain = np.array(Xtrain)

In [ ]:
###################Importing Test Dataset##########################

test = pd.read_csv("Preprocess_Test.csv", header=0)
test = test[test['text'].notnull()]
test

In [ ]:
Xtest = test['text']
Ytest = test['target']

In [ ]:
###################Tokenizing & Padding of Test Dataset##########################

Xtest = '[CLS]' + Xtest + '[SEP]'
Xtest = list(map(tokenizer.tokenize, Xtest))
Xtest = [tokenizer.convert_tokens_to_ids(txt) for txt in Xtest]
Xtest = pad_sequences(Xtest, padding='post', maxlen=maxlen)
Xtest = np.array(Xtest)

# Glove Embedding

In [ ]:
###################Loading Pretrained Word Embeddings to Create Feature Matrix##########################

from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('data_embedding/glove/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for index in range(len(c)):
    for word in c[index]:
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

# Creating the Model Using Deep Learning Techniques for Text Classification

In [ ]:
pooled_outputs1 = []
pooled_outputs2 = []
pooled_outputs3 = []

num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

embedding_dim = 100
embed_input = Input(shape=(maxlen,))

dropout=0.1
filter_sizes = [3,4,5,6]
num_filters = [60,80,100,200]

###################Embedding Layer##########################

x = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen)(embed_input)

###################Attention Layer##########################

for i in range(1):
    att_output = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(x, x)
    att_output = Dropout(dropout)(att_output, training=False)
    layer1_output = LayerNormalization(epsilon=1e-6)(att_output + x)
    ff1_output = Sequential([Dense(ff_dim, activation="relu"),Dense(embedding_dim)])(layer1_output)
    ff1_output = Dropout(dropout)(ff1_output, training=False)
    t = LayerNormalization(epsilon=1e-6)(ff1_output + layer1_output)
    
    pooled_outputs2.append(t)

merge2 = concatenate(pooled_outputs2)

###################CNN Layers##########################
for j in range(len(filter_sizes)):
    for i in range(1):
        conv = (Convolution1D(filters=num_filters[i],
                              kernel_size=filter_sizes[i],
                              padding="same",
                              activation="relu"))(merge2)
        conv = (Convolution1D(filters=num_filters[i+1],
                              kernel_size=filter_sizes[i+1],
                              padding="same",
                              activation="relu"))(conv)
        conv = (Convolution1D(filters=num_filters[i+2],
                              kernel_size=filter_sizes[i+2],
                              padding="same",
                              activation="relu"))(conv)
        conv = (Convolution1D(filters=num_filters[i+3],
                              kernel_size=filter_sizes[i+3],
                              padding="same",
                              activation="relu"))(conv)
        pooled_outputs1.append(conv)
    
merge1 = concatenate(pooled_outputs1)

###################LSTM Layers##########################

# x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(merge1)

x = Flatten()(merge1)
x = Dense(units=1, activation='sigmoid')(x)

model = Model(inputs=[embed_input] , outputs=[x])

# model.summary()

In [ ]:
import pydot
import pydotplus
import tensorflow as tf
from pydotplus import graphviz
from keras.utils.vis_utils import plot_model
from keras.utils.vis_utils import model_to_dot

tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=80
)

In [ ]:
###################Compiling our Model##########################

opt = tf.keras.optimizers.Adam(learning_rate=8e-4)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
###################Training our Model##########################

plrty = ['0', '1']
result = []
preds = []

for i in range(20):
    print(i, ':')
    history = model.fit(Xtrain, Ytrain, validation_split=0.2, epochs=1, batch_size=512)
    preds.append(np.round(model.predict(Xtest)))
#     print(classification_report(Ytest, preds, target_names=plrty))
    results = f1_score(Ytest, preds[i], average='macro')
    print(i, 'result :',results)
    result.append(results)
    
print(result)

In [ ]:
# import numpy as np
# np.savetxt("submission-1.csv", preds, delimiter=",", fmt="%d")